## Python绘制足球传递网络图

***网络图Graph***：
* 由节点、边、权重所组成的数据关系图
* 直观的展示事物节点间的复杂关系

In [1]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Graph, Page

### 1. 读取足球赛队员传球数据集

In [2]:
dataset = pd.read_csv('./datas/passingevents.csv')
dataset.head()

,MatchID,TeamID,OriginPlayerID,DestinationPlayerID,MatchPeriod,EventTime,EventSubType,EventOrigin_x,EventOrigin_y,EventDestination_x,EventDestination_y
0,1,Huskies,Huskies_D1,Huskies_F1,1H,46.323501,Head pass,34,97,59.0,95.0
1,1,Huskies,Huskies_M1,Huskies_F2,1H,51.022546,Simple pass,53,89,69.0,91.0
2,1,Opponent1,Opponent1_D2,Opponent1_G1,1H,89.008721,Simple pass,19,16,5.0,50.0
3,1,Opponent1,Opponent1_G1,Opponent1_F1,1H,92.216160,Launch,5,50,67.0,44.0
4,1,Huskies,Huskies_M2,Huskies_M3,1H,98.265191,Simple pass,42,55,36.0,54.0


In [3]:
# 只取一个场次，一个队伍的数据
dataset = dataset[(dataset["MatchID"]==1) & (dataset["TeamID"]=='Huskies')]
dataset.head()

,MatchID,TeamID,OriginPlayerID,DestinationPlayerID,MatchPeriod,EventTime,EventSubType,EventOrigin_x,EventOrigin_y,EventDestination_x,EventDestination_y
0,1,Huskies,Huskies_D1,Huskies_F1,1H,46.323501,Head pass,34,97,59.0,95.0
1,1,Huskies,Huskies_M1,Huskies_F2,1H,51.022546,Simple pass,53,89,69.0,91.0
4,1,Huskies,Huskies_M2,Huskies_M3,1H,98.265191,Simple pass,42,55,36.0,54.0
8,1,Huskies,Huskies_D1,Huskies_F1,1H,130.656530,Head pass,34,91,52.0,97.0
11,1,Huskies,Huskies_D1,Huskies_G1,1H,161.221470,Simple pass,14,65,11.0,50.0


In [4]:
# 汇总计算，每个球员给其它球员传了多少次球
dataset = dataset.groupby(["OriginPlayerID", "DestinationPlayerID"]).size().reset_index(name='size')
dataset.head()

,OriginPlayerID,DestinationPlayerID,size
0,Huskies_D1,Huskies_D2,14
1,Huskies_D1,Huskies_D3,8
2,Huskies_D1,Huskies_D4,1
3,Huskies_D1,Huskies_D5,3
4,Huskies_D1,Huskies_F1,3


### 2. 构造Graph所需的nodes、links数据

In [5]:
# 所有去重后的队员ID
players = pd.concat([dataset["OriginPlayerID"], dataset["DestinationPlayerID"]]).unique()
players

array(['Huskies_D1', 'Huskies_D2', 'Huskies_D3', 'Huskies_D4',
       'Huskies_D5', 'Huskies_F1', 'Huskies_F2', 'Huskies_F3',
       'Huskies_G1', 'Huskies_M1', 'Huskies_M2', 'Huskies_M3',
       'Huskies_M4', 'Huskies_M5'], dtype=object)

In [6]:
# 图所需的节点列表
nodes = []
for player in players:
    nodes.append({"name": player, "symbolSize": 10})
nodes[:3]

[{'name': 'Huskies_D1', 'symbolSize': 10},
 {'name': 'Huskies_D2', 'symbolSize': 10},
 {'name': 'Huskies_D3', 'symbolSize': 10}]

In [7]:
# 图所需要的边
links=[]
for index, row in dataset.iterrows():
    links.append({"source": row["OriginPlayerID"], 
                  "target": row["DestinationPlayerID"], 
                  "value": row["size"]})
links[:3]

[{'source': 'Huskies_D1', 'target': 'Huskies_D2', 'value': 14},
 {'source': 'Huskies_D1', 'target': 'Huskies_D3', 'value': 8},
 {'source': 'Huskies_D1', 'target': 'Huskies_D4', 'value': 1}]

### 3. 使用Pyecharts绘制Graph图

In [8]:
graph = (
    Graph()
    .add("", 
        nodes,  # 节点列表
        links,  # 边列表
        repulsion=5000,   #节点之间的斥力因子，值越大则斥力越大
        edge_label=opts.LabelOpts(is_show=True, position="middle", formatter="{c}"), # 在边上显示数据
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="传球网络图"))
)

In [9]:
graph.render_notebook()